Probability through Python code
------

<center><img src="https://static.analyticsvidhya.com/wp-content/uploads/2017/02/17081231/Python3-300x300.png" height="500"/></center>

Code Up `P` 
----

`P` is the traditional name for the Probability function:

$$ P(E) = \frac{outcome\ favorable\ to\ E}{total\ number\ of\ outcomes\ in\ sample\ space}$$

In [1]:
from fractions import Fraction

def P(event, space): 
    "The probability of an event, given a sample space of equiprobable outcomes."
    return Fraction(len(event & space), 
                    len(space))

Also note that I use `Fraction` rather than regular division because I want exact answers like 1/3, not 0.3333333333333333.

> Probability is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible.

Warm-up Problem: Die Roll
-----

What's the probability of rolling an even number with a single six-sided fair die? 

We can define the sample space `D` and the event `even`, and compute the probability:

In [2]:
even      = {   2,    4,    6}
die_faces = {1, 2, 3, 4, 5, 6}

P(even, die_faces)

Fraction(1, 2)

It is good to confirm what we already knew. 😀

### Why does the definition of `P` use `len(event & space)` rather than `len(event)`? 

Because I don't want to count outcomes that were specified in `event` but aren't actually in the sample space. Consider:

In [52]:
even = {2, 4, 6, 8, 10, 12, 14}

P(even, die_faces)

Fraction(1, 2)

Here, `len(event)` is 7 `len(space)` is 6, so if just divided, then `P` would be > 1, which is not right.

The favorable cases are the *intersection* of the event and the space, which in Python is `(event & space)`.

Urn Problems
----

<center><img src="http://www2.stetson.edu/~efriedma/periodictable/jpg/Bernoulli-Jacob.jpg" height="500"/></center>

Around 1700, Jacob Bernoulli wrote about removing colored balls from an urn in his landmark treatise *[Ars Conjectandi](https://en.wikipedia.org/wiki/Ars_Conjectandi)*, and ever since then, explanations of probability have relied on [urn problems](https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=probability%20ball%20urn). 

(You'd think the urns would be empty by now.) 

For example, here is a three-part problem [adapted](http://mathforum.org/library/drmath/view/69151.html)  from mathforum.org:

> An urn contains 23 balls: 8 white, 6 blue, and 9 red.  We select six balls at random (each possible selection is equally likely). What is the probability of each of these possible outcomes:

1. All balls are red
2. 3 are blue, 2 are white, and 1 is red
3. Exactly 4 balls are white


So, an outcome is a set of 6 balls, and the sample space is the set of all possible 6 ball combinations. We'll solve each of the 3 parts using our `P` function, and also using basic arithmetic; that is, *counting*. 

Counting is a bit tricky because:

- We have multiple balls of the same color. 
- An outcome is a *set* of balls, where order doesn't matter, not a *sequence*, where order matters.

To account for the first issue, I'll have 8 different white balls labelled `'W1'` through `'W8'`, rather than having eight balls all labelled `'W'`.  That makes it clear that selecting `'W1'` is different from selecting `'W2'`.

The second issue is handled automatically by the `P` function, but if I want to do calculations by hand, I will sometimes first count the number of *permutations* of balls, then get the number of *combinations* by dividing the number of permutations by *c*!, where *c* is the number of balls in a combination. For example, if I want to choose 2 white balls from the 8 available, there are 8 ways to choose a first white ball and 7 ways to choose a second, and therefore 8 &times; 7 = 56 permutations of two white balls. But there are only 56 / 2 = 28 combinations, because `(W1, W2)` is the same combination as `(W2, W1)`.

We'll start by defining the contents of the urn:

In [4]:
def cross(A, B):
    "The set of ways of concatenating one item from collection A with one from B."
    return {a + b 
            for a in A for b in B}

In [5]:
urn = cross('W', '12345678') | cross('B', '123456') | cross('R', '123456789') 
urn

{'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'R1',
 'R2',
 'R3',
 'R4',
 'R5',
 'R6',
 'R7',
 'R8',
 'R9',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8'}

In [6]:
len(urn)

23

Now we can define the sample space, `U6`, as the set of all 6-ball combinations.  We use `itertools.combinations` to generate the combinations, and then join each combination into a string:

In [7]:
import itertools

def combos(items, n):
    "All combinations of n items; each combo as a concatenated str."
    return {' '.join(combo) 
            for combo in itertools.combinations(items, n)}

U6 = combos(items=urn, 
            n=6)
f"{len(U6):,}"

'100,947'

I don't want to print all 100,947 members of the sample space; let's just peek at a random sample of them:

In [8]:
import random

random.sample(U6, 10)

['R6 R5 W7 R3 R8 R1',
 'R2 W5 R9 B2 R3 R1',
 'B4 W5 R9 B2 R8 B1',
 'W8 R4 B3 R5 W1 W3',
 'B5 B3 B4 W5 R9 R1',
 'B5 W8 B3 W7 R1 W3',
 'B3 B4 W6 R7 R1 W1',
 'B5 R6 R4 B2 R3 W3',
 'R2 W8 R9 R3 R1 B1',
 'R6 B4 W5 W6 R5 W7']

Is 100,947 really the right number of ways of choosing 6 out of 23 items, or  "23 choose 6", as  mathematicians [call it](https://en.wikipedia.org/wiki/Combination)?  

Well, we can choose:

- __any of 23__ for the first item
- __any of 22__ for the second
- and so on...
- __any of 18__ for the sixth item

But we don't care about the ordering of the six items, so we divide the product by 6! (the number of permutations of 6 things) giving us:

$$23 ~\mbox{choose}~ 6 = \frac{23 \cdot 22 \cdot 21 \cdot 20 \cdot 19 \cdot 18}{6!} = 100947$$

Note that $23 \cdot 22 \cdot 21 \cdot 20 \cdot 19 \cdot 18 = 23! \;/\; 17!$, so, generalizing, we can write:

$$n ~\mbox{choose}~ c = \frac{n!}{(n - c)! \cdot c!}$$

And we can translate that to code and verify that 23 choose 6 is 100,947:

In [9]:
from math import factorial

def choose(n, c):
    "Number of ways to choose c items from a list of n items."
    return factorial(n) / (factorial(n - c) * factorial(c))

In [10]:
choose(n=23, 
       c=6)

100947.0

Now we're ready to answer the 4 problems: 

### Urn Problem 1: What's the probability of selecting 6 red balls? 

Go try it!

In [60]:
red6 = {s for s in U6 if s.count('R') == 6}

P(red6, U6)

Fraction(4, 4807)

Let's investigate a bit more. How many ways of getting 6 red balls are there?

In [61]:
len(red6)

84

Why are there 84 ways?  

Because there are 9 red balls in the urn, and we are asking how many ways we can choose 6 of them:

In [62]:
choose(9, 6)

84.0

So the probabilty of 6 red balls is then just 9 choose 6 divided by the size of the sample space:

In [63]:
P(red6, U6) == Fraction(int(choose(9, 6)), 
                        len(U6))

True

### Urn Problem 2: what is the probability of 3 blue, 2 white, and 1 red?

In [64]:
b3w2r1 = {s for s in U6 if
          s.count('B') == 3 and s.count('W') == 2 and s.count('R') == 1}

P(b3w2r1, U6)

Fraction(240, 4807)

We can get the same answer by counting how many ways we can choose 3 out of 6 blues, 2 out of 8 whites, and 1 out of 9 reds, and dividing by the number of possible selections:

In [65]:
P(b3w2r1, U6) == Fraction(int(choose(6, 3) * choose(8, 2) * choose(9, 1)), 
                          len(U6))

True

Here we don't need to divide by any factorials, because `choose` has already accounted for that. 

We can get the same answer by figuring: "there are 6 ways to pick the first blue, 5 ways to pick the second blue, and 4 ways to pick the third; then 8 ways to pick the first white and 7 to pick the second; then 9 ways to pick a red. But the order `'B1, B2, B3'` should count as the same as `'B2, B3, B1'` and all the other orderings; so divide by 3! to account for the permutations of blues, by 2! to account for the permutations of whites, and by 100947 to get a probability:

In [66]:
P(b3w2r1, U6) == Fraction((6 * 5 * 4) * (8* 7) * 9, 
                           factorial(3) * factorial(2) * len(U6))

True

### Urn Problem 3: What is the probability of exactly 4 white balls?

We can interpret this as choosing 4 out of the 8 white balls, and 2 out of the 15 non-white balls. 

Then we can solve it the same three ways:

Pick 1 way and solve it!

In [67]:
w4 = {s for s in U6  
          if s.count('W') == 4}

P(w4, U6)

Fraction(350, 4807)

In [68]:
P(w4, U6) == Fraction(int(choose(8, 4) * choose(15, 2)),
                      len(U6))

True

In [69]:
P(w4, U6) == Fraction((8 * 7 * 6 * 5) * (15 * 14),
                      factorial(4) * factorial(2) * len(U6))

True